Importing all the Required Libraries

In [2]:
import numpy as np
import pandas as pd
import string 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn .svm import SVC
from sklearn.linear_model import LogisticRegression

Loading the Dataset Collected

In [3]:
dataframe = pd.read_csv('dataset.csv')
dataframe.head()

,category,rating,LABEL,text_
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...


Creating a new column named Length for the length of each review

In [16]:
dataframe.dropna(inplace=True)
dataframe['Length'] = dataframe['text_'].apply(len)
print(dataframe[dataframe['LABEL']=='OR'][['text_','Length']].sort_values(by='Length', ascending = False).head().iloc[0].text_)
dataframe.head()

WEAK ON CURRENT SCIENCE.
After seeing it twice, I agree with much (but not all) of the positive five star reviews. Out of respect for those who READ reviews, I'll not repeat everything that I like about the presentation. I found the goofy oversize earrings, hairdo, and facial hair arrangement of Daniel Vitalis, (described as a "Wild Food Expert") distracting. UGH. Ditto for David Wolfe, who had an extremely goofy wild hairdo. On the other hand, Jon Gabriel, described as an "author and weight loss expert" was nicely groomed and a good presenter. His story of personal transformation of a fellow of over 400 pounds (whew) to becoming a jock of normal weight was inspiring. Christiane Northrup preserves her rank as one of America's cutest doctors. A really nice looking woman! Presentations by Dr. Mercola, Jason Vale, Kris Carr, Alejandro Junger were fine. It was disappointing to have Jamie Oliver (so popular in the UK) give Baby Cow Growth Fluid a pass with unscientific but popular ideas on 

,category,rating,LABEL,text_,Length
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor...",75
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I...",80
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...,67
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i...",81
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...,85


Removing Punctuations and Stop Words

In [17]:
def convertmyTxt(rv):
    np = [c for c in rv if c not in string.punctuation]
    np = ''.join(np)
    return [w for w in np.split() if w.lower() not in stopwords.words('english')]

In [18]:
#dataframe['Stopwords'] = convertmyTxt(dataframe['text_'])

KeyboardInterrupt: 

Splitting the data to Training and Testing as 75:25

In [48]:
x_train, x_test, y_train, y_test = train_test_split(dataframe['text_'],dataframe['LABEL'], test_size= 0.25)
print("X Train \n",x_train,"Y Train\n", y_train,"X Test\n", x_test,"Y Test\n", y_test)

X Train 
 19449    Destroyed in 10 minutes. Also, it's a very thi...
34586    This was a gift for a friend and she loved it....
33036    Love the fish to go with it and the wooden par...
22350    This is good for small dogs. The only problem ...
19978    All 3 dogs love these, and they are the best. ...
                               ...                        
8860     Great product and works as it should. Haven't ...
39599    I've had this product for years. The quality i...
36041    Gotta say - I love this figure! One of the mor...
22873    I have had to come back to the original formul...
36645    The jeans fit great and very comfortable. Also...
Name: text_, Length: 30324, dtype: object Y Train
 19449    CG
34586    CG
33036    CG
22350    CG
19978    CG
         ..
8860     OR
39599    CG
36041    OR
22873    CG
36645    OR
Name: LABEL, Length: 30324, dtype: object X Test
 1920     Very easy to use. Quickly put it together. Gre...
18710    I've owned this sander for a couple of y

Creating A Pipeline to fit the Training dataset

In [64]:
pip = Pipeline([
    ('bow',CountVectorizer(analyzer = convertmyTxt)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

Fitting The 75% Training dataset to the Pipeline

In [65]:
pip.fit(x_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function convertmyTxt at 0x000002BA16C5E980>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

Predicting The Label Class for Test Dataset

In [66]:
#test = pd.Series(["Got the 256GB version at 21k and it is really worth the price. Camera can be slightly better but battery backup and other features are really good and worthy for the price paid.","I ordered F 19 pro two times one delivered in May and another in Aug. Only after receiving the second F19 Pro , I realized that first phone packet did not have the the earbuds headphone. I request that I be supplied the earbuds deficient in the first order. Also while confirming the dispatch, pls mention list of supplied items.","Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty","The Oppo F19 Pro+ 5G is a solid mid-range offering that delivers impressive performance and an attractive design"])
svcm = pip.predict(x_test)
print(svcm)

['CG' 'CG' 'CG' ... 'OR' 'CG' 'OR']


Finding The Accuracy and Confusion Matrix of The Model

In [67]:
print('Accuracy of the model: ',str(np.round(accuracy_score(y_test,svcm)*100,2)) + '%')
listt = confusion_matrix(y_test,svcm)
print("TP :",listt[0][0])
print("FP :",listt[0][1])
print("FN :",listt[1][0])
print("TN :",listt[1][1])

Accuracy of the model:  89.79%
TP : 4445
FP : 626
FN : 406
TN : 4631
